# **VeriThoughts: Enabling Automated Verilog Code Generation using Reasoning and Formal Verification**

This is the repository for the VeriThoughts Dataset, the first large scale formally verified Verilog reasoning dataset. This repository contains all of the code necessary to generate VeriThoughts as well as our model training and evaluation code.

Our datasets can be found on HuggingFace: [Link](https://huggingface.co/collections/wilyub/verithoughts-datasets-6826de76e798014f05de6c0f)

Our fine-tuned Verilog models can be found on HuggingFace: [Link](https://huggingface.co/collections/nyu-dice-lab/verithoughts-models-681eead7cd13abeb5957baf3)

In [1]:
!git clone https://github.com/wilyub/VeriThoughts.git

Cloning into 'VeriThoughts'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 121 (delta 54), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (121/121), 1.99 MiB | 6.98 MiB/s, done.
Resolving deltas: 100% (54/54), done.


**Download our evaluation dataset (optional)**

In [2]:
!cd VeriThoughts/evaluation_verithoughts && wget https://huggingface.co/datasets/wilyub/VeriThoughtsBenchmark/resolve/main/hf_benchmark.jsonl


--2025-10-16 01:32:46--  https://huggingface.co/datasets/wilyub/VeriThoughtsBenchmark/resolve/main/hf_benchmark.jsonl
Resolving huggingface.co (huggingface.co)... 18.239.50.80, 18.239.50.16, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.80|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /api/resolve-cache/datasets/wilyub/VeriThoughtsBenchmark/ecae68df53c50b113db57c198465d891fcf7736e/hf_benchmark.jsonl?%2Fdatasets%2Fwilyub%2FVeriThoughtsBenchmark%2Fresolve%2Fmain%2Fhf_benchmark.jsonl=&etag=%229f486e6165e9a635087e3eeaa6990949d927537a%22 [following]
--2025-10-16 01:32:46--  https://huggingface.co/api/resolve-cache/datasets/wilyub/VeriThoughtsBenchmark/ecae68df53c50b113db57c198465d891fcf7736e/hf_benchmark.jsonl?%2Fdatasets%2Fwilyub%2FVeriThoughtsBenchmark%2Fresolve%2Fmain%2Fhf_benchmark.jsonl=&etag=%229f486e6165e9a635087e3eeaa6990949d927537a%22
Reusing existing connection to huggingface.co:443.
HTTP request sent, await

**Install required packages**

In [3]:
!pip install -r VeriThoughts/evaluation_verithoughts/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.0 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 

**Example input file format (test.jsonl):**

Each row corresponds to one verilog generation task.

1.   question: prompt for generating verilog codes (prompt should contain the declaration of input/output ports).
2.   ground_truth (optional): the golden verilog implementation; this is used during evaluation.
3.   generated_verilog: no use here



In [1]:
benchmark_path = "/content/VeriThoughts/evaluation_verithoughts/test.jsonl"
#benchmark_path = "/content/VeriThoughts/evaluation_verithoughts/hf_benchmark.jsonl"

In [2]:
import pandas as pd
df = pd.read_json(benchmark_path, lines=True)
df

,ground_truth,question,generated_verilog,verified
0,module bin_to_bcd #(parameter WIDTH = 5) (\n ...,Write a synthesizable Verilog module that conv...,module bin_to_bcd #(parameter WIDTH = 5) (\n ...,True
1,module dice_roller (\n input wire cl...,Write a Verilog module that outputs a pseudo-r...,module dice_roller (\n input wire cl...,True
2,"module seqdet_1011 (\n input wire clk,\n ...",Implement an overlapping sequence detector for...,"module seqdet_1011 (\n input wire clk,\n ...",True
3,module shift_reg #(parameter WIDTH=8) (\n i...,Create a parameterized bidirectional shift reg...,module shift_reg #(parameter WIDTH=8) (\n i...,True
4,module token_bucket #(\n parameter integer ...,Implement a parameterized token bucket rate li...,module token_bucket #(\n parameter integer ...,True
5,module bin_to_bcd #(parameter WIDTH = 5) (\n ...,Write a Verilog module that takes a 5-bit bina...,module bin_to_bcd #(parameter WIDTH = 5) (\n ...,False


In [3]:
model_id = "nyu-dice-lab/Qwen-2.5-Instruct-Verilog-Reasoning-7B"
hf_token = "INSERT-TOKEN"

**Query VeriThoughts LLMs**:


1.   verilog_vllm_multi.py: batch processing verilog generation prompts in the input benchmark file.
2.   model_id: three types of VeriThoughts LLMs (https://huggingface.co/collections/nyu-dice-lab/verithoughts-models-681eead7cd13abeb5957baf3)
3.   sample_number: due to the non-deterministic generation results of LLM, we can query LLMs sample_number times for a single prompt.
4.   batch_size: vllm processes batch_size prompt concurrently.
5.   reasoning_mode: output the reasoning trace.
6.   hf_read_token: your hugging face token.
7.   tensor_parallel_size: number of gpus you can use.



In [4]:
!cd VeriThoughts/evaluation_verithoughts/ && python verilog_vllm_multi.py --model_id nyu-dice-lab/Qwen-2.5-Instruct-Verilog-Reasoning-7B --sample_number 1 --batch_size 1 --reasoning_mode --hf_read_token $hf_token --benchmark_path $benchmark_path --tensor_parallel_size 1

2025-10-16 01:36:41.695031: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-16 01:36:41.711765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760578601.733332    3093 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760578601.739731    3093 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760578601.756068    3093 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [5]:
results_path = "/content/VeriThoughts/evaluation_verithoughts/benchmark_results/nyu-dice-lab/Qwen-2.5-Instruct-Verilog-Reasoning-7B/results.jsonl"

In [6]:
import pandas as pd
result = pd.read_json(results_path, lines=True)
result

,question,full_response,generated_code,ground_truth
0,Write a synthesizable Verilog module that conv...,"Okay, I need to write a Verilog module that co...",\nmodule binary_to_bcd (\n input [4:0] bina...,module bin_to_bcd #(parameter WIDTH = 5) (\n ...
1,Write a Verilog module that outputs a pseudo-r...,"Okay, I need to write a Verilog module called ...","\nmodule random_dig( roll, pipe_sel, valid, ty...",module dice_roller (\n input wire cl...
2,Implement an overlapping sequence detector for...,"Okay, I need to create a Mealy machine for an ...",\nmodule overlapping_sequence_detector (\n in...,"module seqdet_1011 (\n input wire clk,\n ..."
3,Create a parameterized bidirectional shift reg...,"Okay, I need to create a parameterized bidirec...",\nmodule shift_reg #(parameter WIDTH = 8) (\n ...,module shift_reg #(parameter WIDTH=8) (\n i...
4,Implement a parameterized token bucket rate li...,"Okay, I need to create a parameterized Verilog...",\n\nmodule TBUF_RATE_LIMIT #(\n parameter R...,module token_bucket #(\n parameter integer ...


In [ ]:
!zip -r VeriThoughts.zip VeriThoughts


**Example output file format (result.jsonl):**



1.   question: original prompt.
2.   full_response
3.   generated_code: extracted verilog code from full_response.